# Prime Numbered Dice

Author: Mark Richards

Email: mark.thomas.richards@outlook.com

This is a fun little problem I came up with based on dice with prime numbers on them instead of the usual 1 through 6.

The scenario is this:
- There is a bag of dice and each die has a random set of prime numbers on it.
- A person selects a die from the bag at random.
- They then conduct n trials (in secret) where for each trial they roll the dice twice, multiply the numbers, and write down the result.
- Finally they give us the list of results. 
- Our objective is work out just from the list of results what numbers they had on their die. 

There is of course a very __simple and obvious solution__ to this: since the numbers on the die are primes, the result of each trial has a unique factorisation back into primes and we could find this fairly easily. But I want to try to solve this without using prime factorisation and only using Pandas. 

We need to generate some actual data to work on, so below are some functions which produce different data each time.

In [162]:
# Lets take care of the dice generator first. 
# We'll say that each die will have unique values chosen at random from the 
# first 100 primes. 

import math
import random
import pandas as pd

def dice_generator():
    """Generates a 'die' by choosing 6 unique random prime numbers."""
    primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 
              59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 
              127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 
              191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 
              257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 
              331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 
              401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 
              467, 479, 487, 491, 499, 503, 509, 521, 523, 541]
    
    return random.sample(primes, 6)

In [163]:
# Now lets set up the trials.

def run_trials(n=1000):
    """Run n trials using dice_generator()."""
    result = []
    die = dice_generator()
    for i in range(0, n):
        first = random.choice(die)
        second = random.choice(die)
        result.append(first * second)
    
    # Return a series of the result, plus the die as a secret
    return (pd.Series(result), sorted(die))

Now we can run this to generate data. We will not look at the secret die, but will keep it to check our results later.

In [164]:
s_data, secret = run_trials()

Lets look at what we get.

In [165]:
s_data

0         121
1      167897
2       38411
3      205039
4       26909
        ...  
995     38411
996     23119
997      5041
998      4873
999      4331
Length: 1000, dtype: int64

So how do we solve this problem? 

There are `6*6=36` possible outcomes from rolling two dice, but only `((n*n)+n)/2` unique results if we disregard ordering (which does not matter, since the two values are multiplied), and this comes to 21, so we should only have 21 unique values in the data. Lets check.

In [166]:
len(s_data.value_counts())

21

Yes, we do. 

Lets try to visualise some simplified example to see if we can find out how to solve this.

This is the result of rolling a die with sides 2, 3, 5, 7, 11, 13:

In [167]:
df_simplified_example = pd.read_csv("simplified_example.csv", index_col=0)

df_simplified_example

,2,3,5,7,11,13
Primes,,,,,,
2,4,6,10,14,22,26
3,6,9,15,21,33,39
5,10,15,25,35,55,65
7,14,21,35,49,77,91
11,22,33,55,77,121,143
13,26,39,65,91,143,169


The probability of each individual entry in the table is 1/36 but some values are duplicated (mirrored along the top left to bottom right diagonal), so there are only 21 unique outcome values, with differing probabilities.

In fact all of the possible 21 outcome values are mirrored _except for those along the diagonal itself_. These values only appear once and are the result of rolling the same number twice.

If these values only appear once, and they are the result of rolling the same number twice, then if we can identify the equivalent of these numbers in the data, we can find the original dice values by taking their square roots.

As these values only have a 1/36 chance of occuring and all the other values have a 2/36 chance (due to being mirrored), then if we look at how many times each value occurs in the data, the 6 values with the lowest frequency should correspond to the diagonal values - although there will be statistical fluctuation as this is a random process, so this is not guaranteed.

We could then take the square roots of the 6 least frequent values and propose these as the values on the die.

Lets write a function for this.

In [168]:
def proposed_die(s):
    """
    Proposes the sqrt of the 6 least frequent 
    values in s to be the values on the die.
    """
    return (s
        .value_counts()
        .reset_index()
        .tail(6)
        .loc[:, "index"]
        .map(math.sqrt)
        .map(int)
        .sort_values()
        .tolist()
    )

proposed_die(s_data)

[11, 61, 71, 379, 443, 541]

Lets compare this to the secret die:

In [169]:
secret

[11, 61, 71, 379, 443, 541]

In [170]:
proposed_die(s_data) == secret

True

So we have found a way to go from the product of two random dice rolls back to the values on that die. 

But as mentioned above there is __statistical variation__ and this is not a certain method. 

Lets see how it performs for various numbers of trials. 

In [171]:
def test_solution():
    """Run differing numbers of trials."""
    for i in range(100, 2000, 100):
        s_data, secret = run_trials(i)
        die = proposed_die(s_data)
        print(f"Number of trials: {i} - Solution Correct: {die==secret} - Proposal: {die} - Actual: {secret}")

test_solution()

Number of trials: 100 - Solution Correct: False - Proposal: [7, 17, 32, 97, 158, 192] - Actual: [7, 17, 97, 151, 167, 383]
Number of trials: 200 - Solution Correct: False - Proposal: [59, 73, 114, 179, 281, 337] - Actual: [59, 73, 167, 179, 281, 337]
Number of trials: 300 - Solution Correct: False - Proposal: [17, 47, 60, 79, 193, 461] - Actual: [17, 47, 79, 191, 193, 461]
Number of trials: 400 - Solution Correct: False - Proposal: [79, 163, 181, 192, 353, 467] - Actual: [79, 97, 163, 181, 353, 467]
Number of trials: 500 - Solution Correct: True - Proposal: [137, 197, 281, 283, 337, 347] - Actual: [137, 197, 281, 283, 337, 347]
Number of trials: 600 - Solution Correct: False - Proposal: [47, 64, 89, 193, 383, 491] - Actual: [47, 79, 89, 193, 383, 491]
Number of trials: 700 - Solution Correct: False - Proposal: [181, 200, 307, 397, 487, 509] - Actual: [79, 181, 307, 397, 487, 509]
Number of trials: 800 - Solution Correct: True - Proposal: [109, 227, 229, 307, 499, 509] - Actual: [109, 2

As the number of trials increases the success of the procedure increases, as we might expect. But we need in excess of 1000 to have a good chance of success.

Note also that this solution only works because the numbers on the die are primes: it will not work if we allow any random integer to be on the die. The reason is that composite numbers can have multiple factors: 12 is `2*6` but also `3*4`. So the probabilities of getting a specific value can differ and this makes it a little more complicated. 